# Load Bridges2 Modules and Set Environment Variables

In [ ]:
%set_env SHELL=/bin/bash

In [ ]:
import sys
sys.path.append("/usr/share/lmod/lmod/init")
sys.path.append("/jet/home/wnw36/.conda/envs/icomse-dft/lib/python3.1/site-packages/")
from env_modules_python import module
module('load', 'intelmpi')
module('load', 'QuantumEspresso')

In [ ]:
import os
os.environ['ASE_ESPRESSO_COMMAND'] = "mpirun -np 6 pw.x -in PREFIX.pwi > PREFIX.pwo"
os.environ['OMP_NUM_THREADS'] = "2"

# My First SCF Calculation

In [ ]:
from ase.io import read, write
from ase.visualize import view
atoms = read("GaAs.cif")
view(atoms, viewer="x3d")

In [ ]:
from ase.calculators.espresso import Espresso

pseudopotentials = {
    "Ga":"Ga.pbe-dn-kjpaw_psl.1.0.0.UPF",
    "As": "As.nc.z_15.oncvpsp3.dojo.v4-std.upf"}

input_data = {
    'system': {
        'ecutwfc': 43,
        'ecutrho': 300
    },
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': '../pseudos'
    },
    'electrons': {
        'diagonalization':'david',
        'conv_thr': 1.0e-8, 
        'mixing_beta':0.4
    }
} 


calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True, tprnfor=True, 
    input_data = input_data,
    kpts=(2, 2, 2),
    koffset=(1, 1, 1))

atoms.calc = calc


In [ ]:
energy = atoms.get_potential_energy()
print(f"E = {energy:.3f}")

# K-Point Covnergence

In [ ]:
kpoints = range(1,9) 
energies = []
for k in kpoints:
    calc.set(kpts=(k, k, k))
    energies.append(atoms.get_potential_energy())
    

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

plt.plot(kpoints, energies)
plt.xlabel("No. K-Points")
plt.ylabel("E (eV)")
plt.tight_layout()
plt.show()

In [ ]:
calc.set(kpts = (3,3,3))

# Equation of State Fitting

## Automatic

In [ ]:
from ase.eos import EquationOfState, calculate_eos
eos = calculate_eos(atoms, trajectory='GaAs_SJ.traj', npoints=5, eps=0.04)
v, e, B = eos.fit()
eos.plot()
pass

In [ ]:
from ase import units
print((4 * v)**(1 / 3.0), "Ang.")
print(B / units.kJ * 1.0e24, 'GPa')

## Manual

In [ ]:
from ase.io.trajectory import Trajectory
import numpy as np

atoms = read("GaAs.cif")
cell = atoms.get_cell()
atoms.calc = calc

traj = Trajectory('GaAs_murnaghan.traj', 'w')
for x in np.linspace(0.95, 1.05, 5):
    atoms.set_cell(cell * x, scale_atoms=True)
    atoms.get_potential_energy()
    traj.write(atoms)
    

See full list of EOS options at https://databases.fysik.dtu.dk/ase/ase/eos.html

In [ ]:
configs = read("GaAs_murnaghan.traj@0:5")
volumes = [ag.get_volume() for ag in configs]
energies = [ag.get_potential_energy() for ag in configs]
eos = EquationOfState(volumes, energies, eos="murnaghan")
v0, e0, B = eos.fit()
print(B / units.kJ * 1.0e24, 'GPa')
eos.plot()
pass

# Density of States

In [ ]:
from ase.dft.dos import DOS
atoms = read("GaAs.cif")
cell = atoms.get_cell()
atoms.calc = calc
dos = DOS(calc, width=0.2)
d = dos.get_dos()
e = dos.get_energies()

In [ ]:
plt.plot(e, d)
plt.xlabel('energy [eV]')
plt.ylabel('DOS')
plt.show()

In [ ]:
from ase.dft import get_distribution_moment
volume = get_distribution_moment(e,d)
center, width = get_distribution_moment(e,d,(1,2))
print(center, "+/-", width)

# Band Structure

In [ ]:
atoms = read("GaAs.cif")
atoms.calc = calc
atoms.get_potential_energy()
fermi_level = calc.get_fermi_level()

In [ ]:
atoms.get_cell()

In [ ]:
lat = atoms.cell.get_bravais_lattice()
print(lat.description())
lat.plot_bz(show=True)

In [ ]:
kpath = atoms.cell.bandpath()
kpath

In [ ]:
input_data['control'].update(
    {
        'calculation':'bands',
        'restart_mode':'restart',
        'verbosity':'high'
    }
)
calc.set(
    kpts=kpath,
    input_data=input_data
)

calc.calculate(atoms)


In [ ]:
bs = calc.band_structure()
#bs.reference = fermi_level
bs.plot()